In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementClickInterceptedException, JavascriptException, NoSuchElementException
import time
from datetime import date, datetime, timedelta 
import numpy as np
import pandas as pd



In [51]:
for i in range(10):
    print(np.random.rand(1)*4)

[0.74147957]
[0.66160508]
[3.14297709]
[2.9927166]
[2.28143177]
[3.15794266]
[0.40870888]
[3.78185887]
[3.57332194]
[0.77974959]


### Waits for the Pick-Up Location inside the search-widget-form to load

In [2]:

# fixes deprecation error https://stackoverflow.com/questions/64717302/deprecationwarning-executable-path-has-been-deprecated-selenium-python

s=Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=s)
#browser = webdriver.Chrome(executable_path = "C:\Program Files\Chromedriver\chromedriver.exe")
url = 'https://www.campertravelusa.com/'
browser.get(url)



# waits for page elements to load https://stackoverflow.com/questions/26566799/wait-until-page-is-loaded-with-selenium-webdriver-for-python
delay = 3 # seconds
try:
    myElem = WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.XPATH,"//input[contains(@id, 'pickupLocation')]")))
    print ("Page is ready!")
except TimeoutException:
    print ("Loading took too much time!")



#browser.find_element(By.XPATH,"//input[contains(@id, 'pickupLocation')]").send_keys('Orlando')

#//*[@id="search_name"]

#for city in cities:
#browser.find_element(By.XPATH,"//input[contains(@id, 'pickupLocation')]").click()#send_keys(city['city'])
    #browser.find_element(By.XPATH,f"//div[@class = 'autocomplete-suggestion'][@data-val = '{city['city']}']").click()


[WDM] - Downloading: 100%|██████████| 6.21M/6.21M [00:01<00:00, 5.47MB/s]


Page is ready!


### Step 1: Select Pick-up Location and Driver's License


In [5]:
browser.find_element(By.XPATH,"//input[contains(@id, 'pickupLocation')]").clear()
browser.find_element(By.XPATH,"//input[contains(@id, 'pickupLocation')]").click()
time.sleep(int(np.random.rand(1)*5))

browser.find_element(By.XPATH,"//input[contains(@id, 'pickupLocation')]").send_keys('Los Angeles')
time.sleep(int(np.random.rand(1)*5))
browser.find_element(By.XPATH,"//*[@id='c1-5-2-widget-pickupLocation']").send_keys(Keys.RETURN)
time.sleep(int(np.random.rand(1)*5))

#picks the driver's license
browser.find_element(By.CSS_SELECTOR, 'input.form-control.X-CountryOfResidence.AutoCompleteSelectInput').click()
time.sleep(int(np.random.rand(1)*5))
browser.find_element(By.XPATH, "//div[@class = 'autocomplete-suggestion'][@data-val = 'International']").click()

### Step 2: Enter pickup and drop off dates

In [6]:
#create a trip itinerary with start and end dates, trips are 17 days long because that's the average international booking
trip_day = date(2023, 1, 18)
trip = {}
trip['start_month'] = 'Jan'
trip['start_day'] = trip_day.strftime("%d")
trip['end_day'] = (trip_day + timedelta(17)).strftime("%d").lstrip('0')

In [7]:
#open the calendar
browser.find_element(By.XPATH, "//*[@id='c1-5-2-widget-pickupDate']").click()
time.sleep(int(np.random.rand(1)*7))
cal_month = browser.find_element(By.XPATH,"//div[@class = 'dr-cal-start']/div/div/div").find_element(By.CSS_SELECTOR,'button.dp-cal-month').text[0:3]

In [8]:
#Find the month where the trip starts
while cal_month != trip['start_month']:
    browser.find_element(By.XPATH, "//div[@class = 'dr-cal-end']/div/div/div/header/button[@class = 'dp-next']").click()
    #updates cal_month variable to whatever date you are currently on
    cal_month = browser.find_element(By.XPATH,"//div[@class = 'dr-cal-start']/div/div/div").find_element(By.CSS_SELECTOR,'button.dp-cal-month').text[0:3]
    time.sleep(int(np.random.rand(1)*3))
    print(cal_month)

Sep
Oct
Nov
Dec
Jan


In [9]:
#picks the start day of the trip
pu_d = browser.find_element(By.XPATH, f"//div[@class = 'dr-cal-start']/div/div/div/div/button[not(contains(@class, 'edge-day'))][text() = {trip['start_day']}]")
pu_d.click()

In [10]:
#picks the end day, if the start day is smaller than the end day the first page is picked, otherwise the next month's page is picked
if int(trip['start_day']) < int(trip['end_day']):
    do_d = browser.find_element(By.XPATH, f"//div[@class = 'dr-cal-start']/div/div/div/div/button[not(contains(@class, 'edge-day'))][text() = {trip['end_day']}]")
else:
    do_d = browser.find_element(By.XPATH, f"//div[@class = 'dr-cal-end']/div/div/div/div/button[not(contains(@class, 'edge-day'))][text() = {trip['end_day']}]")
do_d.click()

In [11]:
#click the search button
time.sleep(int(np.random.rand(1)*5))
browser.find_element(By.CSS_SELECTOR, 'button.btn.btn-success.btn-lg.btn-block.X-SearchButton').click()



### Step 3: Collect all the prices for the given trip

### CSS Selector Notes

1. CSS_SELECTOR SYNTAX: tagName[AttributeName = 'AttributeValue']

                EXAMPLE: input[id='first_name']

2. If the web element has the CLASS attribute you can use CLASS attribute details in CSS selector. CLASS attribute in CSS SELECTOR is symbolised by the DOT (.) sign  
                   
                SYNTAX : tagName.elementID
                   
                EXAMPLE: input.signup


3. ID attribute in CSS Selector is symbolised by the # sign.
                   
                SYNTAX: tagName#elementID
                EXAMPLE: input#first_name

4. Using mix of ID or CLASSNAME and other attributes of webelement. Add brackets witht he attribute name and value to uniquely identify webelement.
  
                SYNTAX: tagName.classValue[AttributeName='AttributeValue']
                EXAMPLES: input.signup[type='submit'][value = 'Sign me up']
                input#email[placeholder = 'Your email address'][name = 'user[email]']
5. Sub-string matches are very helpful in identifying dynamic webelementswith the help of partial string matches. 
                 
                
                ^ Sign - Signify's the prefix of the text
                $ Sign - Signify's the suffix of the text
                * Sign - Signify's the sub-string of the text

                EXAMPLES:
                Match prefix of text
                input[name^='country_c']

                Match suffix of text
                input[name$='y_client']

                Match sub-string of the text
                    input[name*='try_cl']

6. Finding Child or SubChild Elements
            
                Direct Child - Child comhinator (>) is used to select direct child
                SYNTAX: tagName[AttributeName = 'AttributeValue'] > tagName[AttributeName = 'AttributeValue']
                EXAMPLES: select#country > option[value = 'AU']
                          select#job_role>option[value = 'Developer']
                          form#deorg_form>div

                Child or SubChild - Descendant combinator () is used to select Child or Subchild
                SYNTAX: tagName[AttributeName='AttributeValue'] tagName[AttributeName='AttributeValue']
                EXAMPLE: form#deorg_form div

7. Next Sibling Elements - Located using the + operator

                Adjacent sibling combinator (+) separates two CSS selectors and matches the second webelement
                only if it immediately follows the first webelement, and both are the child of the same parent webelement

                SYNTAX: tagName[attributeName='AttributeValue']+tagName[AttributeName='AttributeValue']



In [60]:
#Why doesn't this work?
browser.execute_script("window.scrollTo(500, 0);")

In [12]:
#switch to correct window
# old method that doesn't work >>> browser.windows.current = browser.windows[1]

browser.switch_to.window(browser.window_handles[1])

# switch to first window
#browser.switch_to.window(browser.window_handles[0])

In [29]:
browser.refresh()

In [46]:
# will have to tell script to wait for page to load when refactored to a loop
results = browser.find_elements(By.CLASS_NAME, 'campervan-result')

In [47]:
#total price
results[0].find_element(By.CLASS_NAME, 'X-VPrice-ConvertedFrom-Amount-Dollars').text

'737'

In [48]:
#Vehicle class
results[0].find_element(By.CSS_SELECTOR, 'span.vehicle-name-text').text

'MiniVan'

In [ ]:
#daily price
results[0].find_element(By.CSS_SELECTOR, 'div.quote-daily-price span.X-VPrice-ConvertedFrom-Amount-Dollars').text

In [51]:
#company name
results[0].find_element(By.CSS_SELECTOR,'div.result-supplier img').get_attribute('alt')

'Travellers Autobarn USA'

In [43]:
results[0].find_element(By.CSS_SELECTOR, 'img.vehicle-image')['src'].split('/')[-2].replace('-', ' ').lower().rstrip('us').strip().title()

TypeError: 'WebElement' object is not subscriptable

In [49]:
#daily price
results[0].find_element(By.CSS_SELECTOR, 'div.quote-daily-price span.X-VPrice-ConvertedFrom-Amount-Dollars').text


'40'

In [52]:
deals = []
for result in results:
    total_price = result.find_element(By.CLASS_NAME, 'X-VPrice-ConvertedFrom-Amount-Dollars').text
    daily_price = result.find_element(By.CSS_SELECTOR, 'div.quote-daily-price span.X-VPrice-ConvertedFrom-Amount-Dollars').text
    vehicle_class = result.find_element(By.CSS_SELECTOR, 'span.vehicle-name-text').text
    company_name = result.find_element(By.CSS_SELECTOR,'div.result-supplier img').get_attribute('alt')
    deals.append({
                'total_price':total_price,
                'daily_price': daily_price,
                'vehicle_class':vehicle_class,
                'company_name':company_name

                })


deals

[{'total_price': '737',
  'daily_price': '40',
  'vehicle_class': 'MiniVan',
  'company_name': 'Travellers Autobarn USA'},
 {'total_price': '888',
  'daily_price': '52',
  'vehicle_class': 'Wanderer',
  'company_name': 'Lost Campers'},
 {'total_price': '952',
  'daily_price': '52',
  'vehicle_class': 'Hitop Campervan',
  'company_name': 'Travellers Autobarn USA'},
 {'total_price': '1025',
  'daily_price': '56',
  'vehicle_class': 'Kuga Campervan',
  'company_name': 'Travellers Autobarn USA'}]

In [79]:
testing = results[0]




WebDriverException: Message: chrome not reachable
  (Session info: chrome=104.0.5112.82)
Stacktrace:
Backtrace:
	Ordinal0 [0x00EA78B3+2193587]
	Ordinal0 [0x00E40681+1771137]
	Ordinal0 [0x00D54070+802928]
	Ordinal0 [0x00D49AB2+760498]
	Ordinal0 [0x00D3CEAF+708271]
	Ordinal0 [0x00D41142+725314]
	Ordinal0 [0x00D455C9+742857]
	Ordinal0 [0x00D55500+808192]
	Ordinal0 [0x00DAD2C2+1168066]
	Ordinal0 [0x00D9D5C6+1103302]
	Ordinal0 [0x00D777E0+948192]
	Ordinal0 [0x00D786E6+952038]
	GetHandleVerifier [0x01150CB2+2738370]
	GetHandleVerifier [0x011421B8+2678216]
	GetHandleVerifier [0x00F317AA+512954]
	GetHandleVerifier [0x00F30856+509030]
	Ordinal0 [0x00E4743B+1799227]
	Ordinal0 [0x00E4BB68+1817448]
	Ordinal0 [0x00E4BC55+1817685]
	Ordinal0 [0x00E55230+1856048]
	BaseThreadInitThunk [0x75F8FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77E17A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77E17A6E+238]


In [41]:
browser.execute_script("document.querySelector('#hbl-live-chat-wrapper').style.display = 'none';")

In [47]:
#before looping through cities we will use a dictionary with only one city
city = {'city':'Los Angeles','state':'CA', 'abbr':'LAX', 'country':'USA'}

#javascript logic, not sure how this works, used to bypass a chat box?
click_chat_js = """
                    var button = document.querySelector('button.olark-launch-button.olark-size-md');
                    function eventFire(elem, etype) {
                      if (elem.fireEvent) {
                        elem.fireEvent('on' + etype);
                      } 
                      else {
                        var evObj = document.createEvent('Events');
                        evObj.initEvent(etype, true, false);
                        elem.dispatchEvent(evObj);
                      }
                    }
                    eventFire(button, 'click');
                """

deals_df = pd.DataFrame()

In [49]:
deals = []
for result in results:

    browser.execute_script("document.querySelector('#hbl-live-chat-wrapper').style.display = 'none';")
    
    if len(result.find_element(By.CSS_SELECTOR, 'span.X-VPrice-ConvertedFrom-Amount')) > 0:
    
        daily_rate = float(result.find_element(By.CSS_SELECTOR, 'span.X-VPrice-ConvertedFrom-Amount').text)
        pu_date = trip['start_date']
        do_date = trip['end_date']
        location = f'{city["city"]}, {city["state"]}'

        if browser.find_element(By.CSS_SELECTOR, 'div#olark-container').isDisplayed():
            browser.execute_script(click_chat_js)

        while True:
            try:
                result.find_by_text('Check Availability').click()
                break
            except ElementClickInterceptedException:
                try:
                    browser.execute_script("window.scrollTo(0, window.scrollY + 200);")
                except JavascriptException:
                    continue
        
        browser.switch_to.window(browser.window_handles[2])

        
        start = time.time()
        while True:
            try:
                if time.time() > start + 30:
                    browser.refresh()
                    # browser.windows.current.close()
                    # browser.windows.current = browser.windows[1]
                    # result.find_by_css('a.button-green.button-book').click()
                    # browser.windows.current = browser.windows[2]
                    start = time.time()
            except TimeoutException:
                continue
            try:
                company = browser.find_element(By.CSS_SELECTOR, 'img.vehicle-image')['src'].split('/')[-2].replace('-', ' ').lower().rstrip('us').strip().title()
                print(company)
                break
            except NoSuchElementException:
                continue
        rv_class = browser.find_element(By.CSS_SELECTOR, 'div.vehicle-name').text
        try:
            section = browser.find_element(By.CSS_SELECTOR,'span.X-VPrice-ConvertedFrom.v-price-total.v-price-converted-from').last
            total = float(section.find_element(By.CSS_SELECTOR, 'span.X-VPrice-ConvertedFrom-Amount').text)
            print(total)
        except NoSuchElementException:
            total = 'Not Listed'

        browser.close()
        browser.switch_to.window(browser.window_handles[1])
        

        deals.append({'Pickup Date':pu_date,'Dropoff Date':do_date,'Class':rv_class,'Company':company,
                    'Daily Rate':daily_rate,'Price':total,'Location':location})

        deals_df = deals_df.append(deals, ignore_index=True)

browser.close()

TypeError: 'WebElement' object is not iterable

In [ ]:
try:
    myElem = WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.XPATH,"//input[contains(@id, 'pickupLocation')]")))
    print ("Page is ready!")
except TimeoutException:
    print ("Loading took too much time!")

#### Function to create a list of trips

In [90]:
# # function to generate list of trips from
# def create_trips(int_stop_date, US):
#     for pu_d in allMondays(int_stop_date):
#         # drop off date is 17 days after today, average internation reservation is 17 days
#         do_d = pu_d + timedelta(17)
#         # for all mondays after today (weeks = 36, is the following year)
#         if pu_d > date.today() + timedelta(10):
#             # if monday is a holiday, change pickup to Tuesday
#             if pu_d in intHolidayClosures(country_holidays, US):
#                 pu_d += timedelta(1)
#             # if Thursday is a holiday, change drop off to Wednesday
#             if do_d in intHolidayClosures(country_holidays, US):
#                 do_d -= timedelta(1)
#             #We should use yield when we want to iterate over a sequence, but don’t want to store the entire sequence in memory. 
#             yield {'start_date':pu_d.strftime('%Y-%m-%d'),
#                     'start_day': pu_d.strftime('%d').lstrip('0'),
#                     'start_month': pu_d.strftime('%b'),
#                     'start_year': pu_d.strftime('%Y'),
#                     'start_date_out':pu_d.strftime('%m-%d-%Y'),
#                     'end_date':do_d.strftime('%Y-%m-%d'),
#                     'end_day':do_d.strftime('%d').lstrip('0'),
#                     'end_month':do_d.strftime('%b'),
#                     'end_year':do_d.strftime('%Y')}

datetime.date(2023, 1, 17)

In [142]:
#click the search button
browser.find_element(By.CSS_SELECTOR, 'button.btn.btn-success.btn-lg.btn-block.X-SearchButton').click()
# browser.windows.current = browser.windows[1]

# while browser.is_element_not_present_by_css('div.campervan-result'):
#     continue

In [143]:
browser.windows.current = browser.windows[1]

AttributeError: 'WebDriver' object has no attribute 'windows'

In [ ]:
cities = [{'city':'San Francisco','state':'CA', 'abbr':'SFO', 'country':'USA'},
          {'city':'Las Vegas','state':'NV', 'abbr':'LAS', 'country':'USA'},
          {'city':'Los Angeles','state':'CA', 'abbr':'LAX', 'country':'USA'},
          {'city':'Seattle','state':'WA', 'abbr':'SEA', 'country':'USA'},
          {'city':'Salt Lake City','state':'UT', 'abbr':'SLC', 'country':'USA'},
          {'city':'Denver','state':'CO', 'abbr':'DEN', 'country':'USA'},
          {'city':'Chicago','state':'IL', 'abbr':'CHI', 'country':'USA'},
          {'city':'Dallas','state':'TX', 'abbr':'DFW', 'country':'USA'},
          {'city':'Orlando','state':'FL', 'abbr':'MCO', 'country':'USA'}]

for city in cities:

    browser.find_element(By.XPATH,"//*[@id='c1-5-2-widget-pickupLocation']").click()
    time.sleep(3)    
  
    browser.find_element(By.XPATH,"//*[@id='c1-5-2-widget-pickupLocation']").send_keys(f"{city['city']}"+ " ")
    time.sleep(3)

    browser.find_element(By.XPATH,"//*[@id='c1-5-2-widget-pickupLocation']").send_keys(Keys.RETURN)